## Get Features

Neded to change to correct environment (problems with linkgrammar)

pip install pyenchant

pip install https://github.com/kpu/kenlm/archive/master.zip

and others...

In [5]:
!bash /home/mlynatom/grammaticality-metrics/heilman-et-al/test.sh /home/mlynatom/grammaticality-metrics/heilman-et-al/sentences.txt

Extracting features from /home/mlynatom/grammaticality-metrics/heilman-et-al/sentences.txt...
Loading the LM will be faster if you build a binary file.
Reading /home/mlynatom/grammaticality-metrics/native_giga.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Loading the LM will be faster if you build a binary file.
Reading /home/mlynatom/grammaticality-metrics/nonnative.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Loading features from /home/mlynatom/grammaticality-metrics/heilman-et-al/sentences.txt
0
Done. features saved to /home/mlynatom/grammaticality-metrics/regression/features/sentences.txt.json
Done


## Regression

In [123]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy import optimize
import numpy as np
from sklearn.linear_model import RidgeCV, Ridge, LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score

from sklearn.model_selection import ParameterGrid
from scipy.stats import pearsonr, kendalltau

from tqdm import tqdm

In [124]:
data = pd.read_json("/home/mlynatom/grammaticality-metrics/heilman-et-al/features/gug_sentences.txt.json", lines=True)

In [125]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   3129 non-null   object 
 1   parse_score          3129 non-null   float64
 2   sentential_top_node  3129 non-null   bool   
 3   dep_count            3129 non-null   int64  
 4   length               3129 non-null   int64  
 5   num_miss             3129 non-null   int64  
 6   prop_miss            3129 non-null   float64
 7   log_miss             3129 non-null   float64
 8   min_s_1              3129 non-null   float64
 9   max_s_1              3129 non-null   float64
 10  sum_s_1              3129 non-null   float64
 11  min_s_2              3117 non-null   float64
 12  max_s_2              3117 non-null   float64
 13  sum_s_2              3117 non-null   float64
 14  min_s_3              3106 non-null   float64
 15  max_s_3              3106 non-null   f

In [126]:
data.head()

,id,parse_score,sentential_top_node,dep_count,length,num_miss,prop_miss,log_miss,min_s_1,max_s_1,...,min_s_2,max_s_2,sum_s_2,min_s_3,max_s_3,sum_s_3,giga_oov,giga_p,toefl11_oov,toefl11_p
0,gug_sentences.txt.0,-6.017418,True,0,21,1,0.05,0.693147,-4.198308,-1.602172,...,-8.126179,-3.946653,-5.603347,-12.163334,-5.006789,-8.242196,2,-2.871818,20,-0.845929
1,gug_sentences.txt.1,-6.424152,True,0,21,2,0.10,1.098612,-4.121595,-1.602172,...,-7.526754,-3.737654,-5.590213,-11.925205,-5.921506,-7.897653,3,-2.701693,20,-0.769989
2,gug_sentences.txt.2,-5.586782,True,0,10,0,0.00,0.000000,-4.269293,-1.602172,...,-7.664845,-3.630444,-5.336554,-8.656352,-6.411362,-7.364122,1,-2.565764,10,-0.999493
3,gug_sentences.txt.3,-9.942500,True,0,4,0,0.00,0.000000,-3.721482,-1.602172,...,-6.397513,-5.235729,-5.828032,-8.804228,-8.780070,-8.792149,1,-4.631810,4,-2.039621
4,gug_sentences.txt.4,-6.999736,True,1,22,0,0.00,0.000000,-4.242428,-1.602172,...,-9.033456,-3.934530,-6.281970,-11.981050,-6.278505,-9.136310,1,-3.156215,22,-0.621265


In [127]:
gug_data = pd.read_csv("/home/mlynatom/grammaticality-metrics/heilman-et-al/gug-data/gug_annotations.tsv", sep="\t")

In [128]:
gug_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3129 entries, 0 to 3128
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Id                       3129 non-null   int64  
 1   Sentence                 3129 non-null   object 
 2   Expert Judgement         3129 non-null   int64  
 3   Crowd Flower Judgements  3129 non-null   object 
 4   Average                  3129 non-null   float64
 5   Dataset                  3129 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 146.8+ KB


In [129]:
gug_data.head()

,Id,Sentence,Expert Judgement,Crowd Flower Judgements,Average,Dataset
0,1,If the teacher once entered in to the class sh...,3,"[3, 3, 3, 4, 2]",3.000000,train
1,2,So I think we can not live if old people could...,2,"[2, 2, 1, 2, 3]",2.000000,test
2,3,New and new technology has been introduced to ...,3,"[3, 4, 3, 3, 2]",3.000000,dev
3,4,For not use car.,1,"[3, 4, 3, 4, 3]",3.000000,test
4,5,Here was no promise of morning except that we ...,3,"[2, 4, 3, 2, 3]",2.833333,test


In [130]:
merged_data = pd.concat([data, gug_data], axis=1, )
display(merged_data.head())

,id,parse_score,sentential_top_node,dep_count,length,num_miss,prop_miss,log_miss,min_s_1,max_s_1,...,giga_oov,giga_p,toefl11_oov,toefl11_p,Id,Sentence,Expert Judgement,Crowd Flower Judgements,Average,Dataset
0,gug_sentences.txt.0,-6.017418,True,0,21,1,0.05,0.693147,-4.198308,-1.602172,...,2,-2.871818,20,-0.845929,1,If the teacher once entered in to the class sh...,3,"[3, 3, 3, 4, 2]",3.000000,train
1,gug_sentences.txt.1,-6.424152,True,0,21,2,0.10,1.098612,-4.121595,-1.602172,...,3,-2.701693,20,-0.769989,2,So I think we can not live if old people could...,2,"[2, 2, 1, 2, 3]",2.000000,test
2,gug_sentences.txt.2,-5.586782,True,0,10,0,0.00,0.000000,-4.269293,-1.602172,...,1,-2.565764,10,-0.999493,3,New and new technology has been introduced to ...,3,"[3, 4, 3, 3, 2]",3.000000,dev
3,gug_sentences.txt.3,-9.942500,True,0,4,0,0.00,0.000000,-3.721482,-1.602172,...,1,-4.631810,4,-2.039621,4,For not use car.,1,"[3, 4, 3, 4, 3]",3.000000,test
4,gug_sentences.txt.4,-6.999736,True,1,22,0,0.00,0.000000,-4.242428,-1.602172,...,1,-3.156215,22,-0.621265,5,Here was no promise of morning except that we ...,3,"[2, 4, 3, 2, 3]",2.833333,test


In [131]:
merged_data = merged_data[merged_data["Expert Judgement"] != 0] #not valid data

In [132]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3019 entries, 0 to 3128
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       3019 non-null   object 
 1   parse_score              3019 non-null   float64
 2   sentential_top_node      3019 non-null   bool   
 3   dep_count                3019 non-null   int64  
 4   length                   3019 non-null   int64  
 5   num_miss                 3019 non-null   int64  
 6   prop_miss                3019 non-null   float64
 7   log_miss                 3019 non-null   float64
 8   min_s_1                  3019 non-null   float64
 9   max_s_1                  3019 non-null   float64
 10  sum_s_1                  3019 non-null   float64
 11  min_s_2                  3016 non-null   float64
 12  max_s_2                  3016 non-null   float64
 13  sum_s_2                  3016 non-null   float64
 14  min_s_3                  3014

In [133]:
train = merged_data[merged_data["Dataset"] == "train"]
val = merged_data[merged_data["Dataset"] == "dev"]
test = merged_data[merged_data["Dataset"] == "test"]

In [134]:
train = train.drop(columns=["id", "Id", "Sentence", "Expert Judgement", "Crowd Flower Judgements", "Dataset"])
val = val.drop(columns=["id", "Id", "Sentence", "Expert Judgement", "Crowd Flower Judgements", "Dataset"])
test = test.drop(columns=["id", "Id", "Sentence", "Expert Judgement", "Crowd Flower Judgements", "Dataset"])

In [135]:
#drop rows with nans
train = train.dropna(axis=0)
val = val.dropna(axis=0)
test = test.dropna(axis=0)

In [136]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1516 entries, 0 to 3127
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   parse_score          1516 non-null   float64
 1   sentential_top_node  1516 non-null   bool   
 2   dep_count            1516 non-null   int64  
 3   length               1516 non-null   int64  
 4   num_miss             1516 non-null   int64  
 5   prop_miss            1516 non-null   float64
 6   log_miss             1516 non-null   float64
 7   min_s_1              1516 non-null   float64
 8   max_s_1              1516 non-null   float64
 9   sum_s_1              1516 non-null   float64
 10  min_s_2              1516 non-null   float64
 11  max_s_2              1516 non-null   float64
 12  sum_s_2              1516 non-null   float64
 13  min_s_3              1516 non-null   float64
 14  max_s_3              1516 non-null   float64
 15  sum_s_3              1516 non-null   float6

In [137]:
X_train = train.drop("Average", axis=1)
y_train = train["Average"]

X_val = val.drop("Average", axis=1)
y_val = val["Average"]

X_test = test.drop("Average", axis=1)
y_test = test["Average"]

In [138]:
def apply_scaling(train: pd.DataFrame, val: pd.DataFrame, test: pd.DataFrame, scaler):
    X_train_scaled = train
    X_val_scaled = val
    X_test_scaled = test
    if scaler is not None:
        # init scaler
        scaler = scaler()
        X_train_scaled = scaler.fit_transform(train)
        X_val_scaled = scaler.transform(val)
        X_test_scaled = scaler.transform(test)

    return X_train_scaled, X_val_scaled, X_test_scaled

In [139]:
X_train_scaled, X_val_scaled, X_test_scaled = apply_scaling(X_train, X_val, X_test, StandardScaler)

In [140]:
def get_opt_ridge_model(Xtrain, Xval, ytrain, yval):
    def ridgemodel_eval(alpha):
        clf = Ridge(alpha=alpha)
        clf.fit(Xtrain, ytrain)
        return mean_squared_error(yval, clf.predict(Xval), squared = False)

    # Find Ridge alpha automatically
    opt_alpha = optimize.minimize_scalar(ridgemodel_eval, options = {'maxiter': 30}, method = 'bounded', bounds=(0.1, 400))
    print('Optimal alpha', opt_alpha)
    print("\n")

    clf_opt_ridge = Ridge(alpha = opt_alpha.x)
    clf_opt_ridge.fit(Xtrain, ytrain)
    return clf_opt_ridge

In [141]:
opt_model = get_opt_ridge_model(X_train, X_val, y_train, y_val)

Optimal alpha  message: Solution found.
 success: True
  status: 0
     fun: 0.4842792486930455
       x: 6.784724038741232
     nit: 17
    nfev: 17




In [142]:
preds = opt_model.predict(X_test)

In [143]:
pearsonr(preds, y_test)

PearsonRResult(statistic=0.5147811722921918, pvalue=4.063340309715265e-52)

In [144]:
kendalltau(preds, y_test)

SignificanceResult(statistic=0.34978430975957125, pvalue=4.9695320518203904e-43)

### binarize

In [145]:
def binarize(lst):
    return [1 if avg > 3.5 else 0 for avg in lst]

In [146]:
bin_preds_ridge = binarize(preds)

In [147]:
y_train_bin = binarize(y_train)
y_val_bin = binarize(y_val)
y_test_bin = binarize(y_test)

In [161]:
np.unique(bin_preds_ridge, return_counts=True)

(array([0, 1]), array([751,   1]))

In [148]:
accuracy_score(y_test_bin, bin_preds_ridge)

0.8457446808510638

In [149]:
kendalltau(y_test_bin, bin_preds_ridge)

SignificanceResult(statistic=0.0850107726184491, pvalue=0.0198237341873847)

## LOGREG

In [150]:
parameters_grid = ParameterGrid({"l1_ratio": list(np.arange(0, 1, 0.01))})

In [151]:
acc = []

for params in tqdm(parameters_grid):
    tmp_logreg = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=1000, **params)
    tmp_logreg.fit(X_train_scaled, y_train_bin)
    preds = tmp_logreg.predict(X_val_scaled)
    acc.append(accuracy_score(y_val_bin, preds))

100%|██████████| 100/100 [00:15<00:00,  6.26it/s]


In [152]:
best_l1 = parameters_grid[np.argmax(acc)]["l1_ratio"]

In [153]:
best_l1

0.0

In [154]:
logreg = LogisticRegression(penalty='elasticnet', solver='saga', max_iter=10000, l1_ratio=best_l1)
logreg_clf = logreg.fit(X_train, y_train_bin)

In [175]:
logreg = LogisticRegression(penalty='l2', solver='saga', max_iter=10000)
logreg_clf = logreg.fit(X_train, y_train_bin)

In [176]:
preds_logreg = logreg_clf.predict(X_test)

In [177]:
np.unique(preds_logreg, return_counts=True)

(array([0, 1]), array([751,   1]))

In [178]:
accuracy_score(y_test_bin, preds_logreg)

0.8457446808510638

In [179]:
kendalltau(y_test_bin, preds_logreg)

SignificanceResult(statistic=0.0850107726184491, pvalue=0.0198237341873847)

In [180]:
logreg_proba = logreg_clf.predict_proba(X_test)[:, 1]

In [182]:
pearsonr(logreg_proba, y_test)

PearsonRResult(statistic=0.45359650306338406, pvalue=1.9471337225020114e-39)

In [183]:
kendalltau(logreg_proba, y_test)

SignificanceResult(statistic=0.34611390565949735, pvalue=3.6153055993069564e-42)